# Scrapper Banco de Chile - Foo Monkeys

### Importaciones

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import os

### Scraping

Ponemos la url con la que se trabajará.

In [2]:
url = 'https://portales.bancochile.cl/personas/beneficios/contenidos.json?page='

Buscamos en cada página que haya y guardamos todos los descuentos en una lista.

In [3]:
# set the driver
driver = webdriver.Chrome()
driver.maximize_window()

# create data list of pages
discounts = []

# set paging
totalPages = 1
currentPage = 1

# begin loop
while currentPage <= totalPages:

    # get page currentPage
    driver.get(f'{url}{currentPage}')
    pageSource = driver.page_source

    # process it
    soup = BeautifulSoup(pageSource, 'html.parser')
    content = soup.find('pre')
    jsonText = content.get_text()
    data = json.loads(jsonText)

    # if this is the first page, use it to set the actual totalPages
    if currentPage == 1:
        totalPages = data['meta']['total_pages']

    discounts.extend(data['contenidos'])
    
    # update currentPage
    currentPage+=1

driver.quit()

Screapeamos la página de detalle de cada descuento, la procesamos y los agregamos al descuento.

In [31]:
# set the driver
driver = webdriver.Chrome()
driver.maximize_window()

# begin loop
for discount in discounts:

    # get url containing the details
    url = discount['url']

    # get the details
    driver.get(url)
    pageSource = driver.page_source

    # get the parts we want to save
    detailDescription = soup.find(class_='detail__description')
    conditions = soup.find('p', class_='info__content beneficios-transversal').get_text(strip=True)
    
    # make an empty dictionary
    details = {} 
    
    # create a detailsList from the elements in detail_description
    detailsList = []
    liElements = detail_description.find_all('li')
    for li in liElements:
        detailsList.append(li.get_text(strip=True))
    
    # add summary, detailsList and conditions to the details dictionary
    details['summary'] = content.find('p').get_text(strip=True)
    details['list'] = detailsList
    details['conditions'] = conditions

    # add the details to the discount
    discount['details'] = details

driver.quit()

In [35]:
filename = 'discountsBancoDeChile.json'
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(discounts, f, ensure_ascii=False, indent=4)